

```
# This is formatted as code
```

Testing model deteksi chihuahua


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# !pip install tensorflow-gpu==2.0.0
# %tensorflow_version 2.0.0

In [3]:
# !pip install keras==2.3.1

In [4]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Root directory dari project
ROOT_DIR = os.path.abspath("/content/drive/MyDrive/Deep Learning/Tugas Kelompok DL/Tugas Implementasi Mask RCNN/Colab/Mask_RCNN-master")
print(ROOT_DIR)

/content/drive/MyDrive/Deep Learning/Tugas Kelompok DL/Tugas Implementasi Mask RCNN/Colab/Mask_RCNN-master


In [7]:
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

In [8]:
class ChihuahuaConfig(Config):
    """Configuration for training on the balloon dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "chihuahua"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + balloon

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

In [9]:
class CihuahuaDataset(utils.Dataset):
  def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "cihuahua":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [10]:
def color_splash(image, mask):
    """Apply color splash effect.
    image: RGB image [height, width, 3]
    mask: instance segmentation mask [height, width, instance count]

    Returns result image.
    """
    # Make a grayscale copy of the image. The grayscale copy still
    # has 3 RGB channels, though.
    gray = skimage.color.gray2rgb(skimage.color.rgb2gray(image)) * 255
    # Copy color pixels from the original color image where mask is set
    if mask.shape[-1] > 0:
        # We're treating all instances as one, so collapse the mask into one layer
        mask = (np.sum(mask, -1, keepdims=True) >= 1)
        splash = np.where(mask, image, gray).astype(np.uint8)
    else:
        splash = gray.astype(np.uint8)
    return splash


def detect_and_color_splash(model, image_path=None, video_path=None):
    assert image_path or video_path

    # Image or video?
    if image_path:
        # Run model detection and generate the color splash effect
        print("Running on {}".format(image_path))
        # Read image
        image = skimage.io.imread(image_path)
        # Detect objects
        r = model.detect([image], verbose=1)[0]
        # Color splash
        splash = color_splash(image, r['masks'])
        # Save output
        file_name = ROOT_DIR + "/splash_{:%Y%m%dT%H%M%S}.png".format(datetime.datetime.now())
        skimage.io.imsave(file_name, splash)
    elif video_path:
        import cv2
        # Video capture
        vcapture = cv2.VideoCapture(video_path)
        width = int(vcapture.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(vcapture.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = vcapture.get(cv2.CAP_PROP_FPS)

        # Define codec and create video writer
        file_name = "splash_{:%Y%m%dT%H%M%S}.avi".format(datetime.datetime.now())
        vwriter = cv2.VideoWriter(file_name,
                                  cv2.VideoWriter_fourcc(*'MJPG'),
                                  fps, (width, height))

        count = 0
        success = True
        while success:
            print("frame: ", count)
            # Read next image
            success, image = vcapture.read()
            if success:
                # OpenCV returns images as BGR, convert to RGB
                image = image[..., ::-1]
                # Detect objects
                r = model.detect([image], verbose=0)[0]
                # Color splash
                splash = color_splash(image, r['masks'])
                # RGB -> BGR to save image to video
                splash = splash[..., ::-1]
                # Add image to video writer
                vwriter.write(splash)
                count += 1
        vwriter.release()
    print("Saved to ", file_name)

In [12]:
#Path Image baloon yang mau di splash
images_path = ROOT_DIR + '/datasets/chihuahua-test.jpg'

#Path Weight baloon, menggunakan weights yang sudah di training
weights_path = ROOT_DIR + '/logs/chihuahua20210518T0511/mask_rcnn_chihuahua_0030.h5'
# weights_path = ROOT_DIR + '/mask_rcnn_balloon.h5'


logs = DEFAULT_LOGS_DIR

class InferenceConfig(ChihuahuaConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()

model = modellib.MaskRCNN(mode="inference", config=config,
                                  model_dir=logs)



# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

#Splash images
detect_and_color_splash(model, image_path=images_path,)
# detect_and_color_splash(model, image_path=images_path,
#                                 video_path=video)

Loading weights  /content/drive/MyDrive/Deep Learning/Tugas Kelompok DL/Tugas Implementasi Mask RCNN/Colab/Mask_RCNN-master/logs/chihuahua20210518T0511/mask_rcnn_chihuahua_0030.h5
Re-starting from epoch 30
Running on /content/drive/MyDrive/Deep Learning/Tugas Kelompok DL/Tugas Implementasi Mask RCNN/Colab/Mask_RCNN-master/datasets/chihuahua-test.jpg
Processing 1 images
image                    shape: (500, 750, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Saved to  /content/drive/MyDrive/Deep Learning/Tugas Kelompok DL/Tugas Implementasi Mask RCNN/Colab/Mask_RCNN-master/splash_20210518T064411.png
